## Import de Librerias

In [90]:
import face_recognition
from extraccion_de_caracteristicas import load_json
import time
import os 


In [91]:
from knn_secuencial import range_search #Pronto integrar aca
from knn_rtree import * #Pronto corregir

## Carga de los encodings del Dataset

In [70]:
dataset = load_json()

## Pruebas

In [113]:
N_values = [100,200,400,800,1600,3200,6400,12800]
k = 8   # fijo
image_path = "./test/teofilo.png"
query_image = face_recognition.load_image_file(image_path)
faces_encoding = face_recognition.face_encodings(query_image)[0]

/home/diego/anaconda3/envs/mainds/lib/python3.9/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


### KNN Sequential

In [110]:
import face_recognition
import heapq as pq

In [114]:
def knn_pq(faces_encoding, dataset, k):
    result = []
    for path, matrix_vector_faces in dataset:
        for distance in face_recognition.face_distance(matrix_vector_faces, faces_encoding):
            #result.append((os.path.basename(path), distance))
            pq.heappush(result, (distance, os.path.basename(path)))

    resultK = heapq.nsmallest(k , result)

    return [os.path.basename(path) for distance, path in resultK]


In [115]:
''' Testeando KNN-Secuencial PQ'''
print("KNN-Secuencial PQ TEST")
for n in N_values:
    print("N =",n)
    start_time = time.time()
    data = dataset[:n]
    print(knn_pq(faces_encoding, data, k))
    end_time = time.time()
    tiempo_ejecucion = end_time - start_time
    print(tiempo_ejecucion)

KNN-Secuencial PQ TEST
N = 100
['Angel_Maza_0001.jpg', 'Anthony_Pico_0001.jpg', 'Gro_Harlem_Brundtland_0001.jpg', 'Reginald_Hudlin_0001.jpg', 'Mike_Smith_0001.jpg', 'Xiang_Huaicheng_0001.jpg', 'Dragan_Covic_0001.jpg', 'Ranil_Wickremasinghe_0001.jpg']
0.0043182373046875
N = 200
['Wilbert_Elki_Meza_Majino_0001.jpg', 'Jeong_Se-hyun_0001.jpg', 'Jeong_Se-hyun_0004.jpg', 'Jeong_Se-hyun_0009.jpg', 'Jeong_Se-hyun_0003.jpg', 'Jeong_Se-hyun_0006.jpg', 'Angel_Maza_0001.jpg', 'Jeong_Se-hyun_0008.jpg']
0.008858919143676758
N = 400
['George_Lopez_0003.jpg', 'Wilbert_Elki_Meza_Majino_0001.jpg', 'Jeong_Se-hyun_0001.jpg', 'Jeong_Se-hyun_0004.jpg', 'Jeong_Se-hyun_0009.jpg', 'Jeong_Se-hyun_0003.jpg', 'Jeong_Se-hyun_0006.jpg', 'George_Lopez_0005.jpg']
0.011796236038208008
N = 800
['George_Lopez_0003.jpg', 'Miguel_Estrada_0001.jpg', 'Gary_Locke_0001.jpg', 'Chen_Liang_Yu_0002.jpg', 'Wilbert_Elki_Meza_Majino_0001.jpg', 'Miguel_Estrada_0002.jpg', 'Jeong_Se-hyun_0001.jpg', 'Hamzah_Haz_0002.jpg']
0.027548313140

In [79]:
def select_representative_radii(mean_distance, std_distance):
    # regla empírica
    r1 = mean_distance - std_distance
    r2 = mean_distance - 2 * std_distance
    r3 = mean_distance - 3 * std_distance

    return [r1, r2, r3]

def distances(dataset):
    vector_dist = []
    for path, matrix_vector_faces in dataset:
        for distance in face_recognition.face_distance(matrix_vector_faces, faces_encoding[0]):
            vector_dist.append(distance)
    print("AQUIIIII")
    print(vector_dist)

    return list((np.mean(vector_dist), np.std(vector_dist)))

distances = distances(dataset)
radios = select_representative_radii(distances[0],distances[1])
print(radios)


AQUIIIII
[1.9006955292008207, 1.9765809169563646, 1.8920508220898298, 1.9883928774690995, 2.0088168344236856, 1.8998937387616595, 1.9385573731819563, 1.8854099966032096, 1.8673168845265329, 1.868806739745065, 1.8793510432703928, 1.909672510429954, 1.904448231466559, 1.8782064171129262, 1.8460457941444026, 1.879281595708346, 1.8948924313463091, 1.841722091496564, 2.0072658456318058, 1.8976171906086639, 1.862491922178827, 1.857812117493347, 1.7966401293390026, 1.7636756324440077, 1.94757724897243, 2.0211894300343807, 2.017902799471207, 2.022143843846772, 1.872409570018676, 1.910569677346002, 1.9604422636718728, 1.9706301070344414, 1.9117538253818276, 2.007006637822222, 1.9718100286999072, 1.9852601265272842, 1.9655286613867575, 1.9329362441729065, 2.047563251472029, 2.069970960799001, 1.9696118458802032, 1.9698906840551027, 1.9557117563706978, 1.9807091886594577, 2.005309152401631, 1.9518637193684127, 1.84934729491252, 1.8726945215771493, 1.9810055757473906, 1.9563773817357906, 1.9995629

In [68]:
''' Testeando KNN-Secuencial RS '''
print("KNN-Secuencial Range Search TEST")
for i in radios:
    print("Testing with r =", i)
    for n in N_values:
        print("N =",n)
        start_time = time.time()
        print(range_search(faces_encoding, dataset[:n], i))
        end_time = time.time()
        tiempo_ejecucion = end_time - start_time
        print(tiempo_ejecucion)

KNN-Secuencial Range Search TEST
Testing with r = 0.7091430536563229
N = 100
['/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Angel_Maza/Angel_Maza_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Anthony_Pico/Anthony_Pico_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Gro_Harlem_Brundtland/Gro_Harlem_Brundtland_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Reginald_Hudlin/Reginald_Hudlin_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Mike_Smith/Mike_Smith_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Xiang_Huaicheng/Xiang_Huaicheng_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Dragan_Covic/Dragan_Covic_0001.jpg', '/Users/ValDLaw/Documents/GitHub

In [41]:
''' Testeando KNN-RTree '''
print("KNN-Rtree TEST")
for n in N_values:
    print("N =",n)
    start_time = time.time()
    print(knn_rtree(faces_encoding, k, dataset[:n]))
    end_time = time.time()
    tiempo_ejecucion = end_time - start_time
    print(tiempo_ejecucion)

KNN-Rtree TEST
N = 100
['/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Claudia_Pechstein/Claudia_Pechstein_0003.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Mike_Smith/Mike_Smith_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Gro_Harlem_Brundtland/Gro_Harlem_Brundtland_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Paul_ONeill/Paul_ONeill_0008.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Ranil_Wickremasinghe/Ranil_Wickremasinghe_0003.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/German_Khan/German_Khan_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Angel_Maza/Angel_Maza_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend